In [2]:
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
import lightgbm as lgb
import time
import pandas as pd
import numpy as np

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
path = '/home/xychen'
# 全量数据

# train = pd.read_table(open((path + '/round1_iflyad_train.txt'),encoding='utf-8'))
# test = pd.read_table(open((path + '/round1_iflyad_test_feature.txt'),encoding='utf-8'))
# pre = pd.concat([train, test], axis=0, ignore_index=True)
# pre.to_csv(path + '/data_all.csv')


In [5]:
data = pd.read_csv(open((path + '/data_all.csv'),encoding='utf-8'))

In [6]:
data.head(5)

,Unnamed: 0,adid,advert_id,advert_industry_inner,advert_name,app_cate_id,app_id,app_paid,campaign_id,carrier,...,make,model,nnt,orderid,os,os_name,osv,province,time,user_tags
0,0,1560128,230000063,102400_102401,B4734117F35EE97F,107.0,2089229.0,False,1000023,1,...,HUAWEI,HUAWEI-CAZ-AL10,1,3010798,2,android,7.0.0,137103102100100,2190219034,NaN
1,1,1488859,230000063,102400_102401,B4734117F35EE97F,108.0,2070079.0,False,1000023,3,...,Xiaomi,Redmi Note 4,1,2311397,2,android,6.0,137105101100100,2190221070,"2100191,2100078,3001825,,3001781,3001791,30017..."
2,2,1537089,230000065,101700_101704,E257895F74792E81,100.0,2089397.0,False,1000021,3,...,OPPO,OPPO+R11s,1,3008491,2,android,7.1.1,137103104100100,2190219793,NaN
3,3,1577884,230001710,101900_101902,0A421D7B11EABFC5,100.0,2071234.0,False,1003544,0,...,NaN,OPPO A57,1,3011304,2,android,6.0.1,137103102100100,2190221704,"2100098,gd_2100000,3001791,3001795,3002193,300..."
4,4,1432367,230000063,102400_102401,B4734117F35EE97F,103.0,1030051.0,False,1000023,1,...,Apple,iPhone 7,3,2304491,1,ios,11.1.1,137103102100100,2190220024,NaN


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041674 entries, 0 to 1041673
Data columns (total 36 columns):
Unnamed: 0               1041674 non-null int64
adid                     1041674 non-null int64
advert_id                1041674 non-null int64
advert_industry_inner    1041674 non-null object
advert_name              1041674 non-null object
app_cate_id              1039376 non-null float64
app_id                   1039376 non-null float64
app_paid                 1041674 non-null bool
campaign_id              1041674 non-null int64
carrier                  1041674 non-null int64
city                     1041674 non-null int64
click                    1001650 non-null float64
creative_has_deeplink    1041674 non-null bool
creative_height          1041674 non-null int64
creative_id              1041674 non-null int64
creative_is_download     1041674 non-null bool
creative_is_js           1041674 non-null bool
creative_is_jump         1041674 non-null bool
creative_is_voicead 

In [8]:
data = data.fillna(-1)
data['day'] = data['time'].apply(lambda x: int(time.strftime("%d", time.localtime(x))))
data['hour'] = data['time'].apply(lambda x: int(time.strftime("%H", time.localtime(x))))
data['label'] = data.click.astype(int)
del data['click']

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041674 entries, 0 to 1041673
Data columns (total 38 columns):
Unnamed: 0               1041674 non-null int64
adid                     1041674 non-null int64
advert_id                1041674 non-null int64
advert_industry_inner    1041674 non-null object
advert_name              1041674 non-null object
app_cate_id              1041674 non-null float64
app_id                   1041674 non-null float64
app_paid                 1041674 non-null bool
campaign_id              1041674 non-null int64
carrier                  1041674 non-null int64
city                     1041674 non-null int64
creative_has_deeplink    1041674 non-null bool
creative_height          1041674 non-null int64
creative_id              1041674 non-null int64
creative_is_download     1041674 non-null bool
creative_is_js           1041674 non-null bool
creative_is_jump         1041674 non-null bool
creative_is_voicead      1041674 non-null bool
creative_tp_dnf        

In [10]:
#首先来一个全局的变量类型转换处理，6个布尔型特征，单独踢出来，处理一下，将布尔型变为int32型。
bool_feature = ['creative_is_jump', 'creative_is_download', 'creative_is_js', 'creative_is_voicead',
                'creative_has_deeplink', 'app_paid']
for i in bool_feature:
    data[i] = data[i].astype(int)   #啥都不写，只写int的话，则默认为int32.

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041674 entries, 0 to 1041673
Data columns (total 38 columns):
Unnamed: 0               1041674 non-null int64
adid                     1041674 non-null int64
advert_id                1041674 non-null int64
advert_industry_inner    1041674 non-null object
advert_name              1041674 non-null object
app_cate_id              1041674 non-null float64
app_id                   1041674 non-null float64
app_paid                 1041674 non-null int64
campaign_id              1041674 non-null int64
carrier                  1041674 non-null int64
city                     1041674 non-null int64
creative_has_deeplink    1041674 non-null int64
creative_height          1041674 non-null int64
creative_id              1041674 non-null int64
creative_is_download     1041674 non-null int64
creative_is_js           1041674 non-null int64
creative_is_jump         1041674 non-null int64
creative_is_voicead      1041674 non-null int64
creative_tp_dnf  

In [12]:
data['advert_industry_inner_1'] = data['advert_industry_inner'].apply(lambda x: x.split('_')[0])

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041674 entries, 0 to 1041673
Data columns (total 39 columns):
Unnamed: 0                 1041674 non-null int64
adid                       1041674 non-null int64
advert_id                  1041674 non-null int64
advert_industry_inner      1041674 non-null object
advert_name                1041674 non-null object
app_cate_id                1041674 non-null float64
app_id                     1041674 non-null float64
app_paid                   1041674 non-null int64
campaign_id                1041674 non-null int64
carrier                    1041674 non-null int64
city                       1041674 non-null int64
creative_has_deeplink      1041674 non-null int64
creative_height            1041674 non-null int64
creative_id                1041674 non-null int64
creative_is_download       1041674 non-null int64
creative_is_js             1041674 non-null int64
creative_is_jump           1041674 non-null int64
creative_is_voicead        1041

In [14]:
#13个与广告有关的特征，一起拿出来处理。成为一个广告特征列表。说明后面肯定会有for循环之类的迭代操作。
#注意此处忽略3个没有用到特征creative_is_js   ，creative_height  ，creative_width  ，我们认为它们比较多余，将之忽略。
#亦可能不是一走上来就知道这么挑选，肯定是历经多次尝试。
ad_cate_feature = ['adid', 'advert_id', 'orderid', 'advert_industry_inner_1', 'advert_industry_inner', 'advert_name',
                   'campaign_id', 'creative_id', 'creative_type', 'creative_tp_dnf', 'creative_has_deeplink',
                   'creative_is_jump', 'creative_is_download']

In [15]:
#4个与媒体有关的特征，一起拿出来处理。成为一个媒体特征列表。说明后面肯定会有for循环之类的迭代操作。
#注意此处忽略2个没有用到特征app_paid，instance_id ，我们认为它们比较多余，将之忽略。
#亦可能不是一走上来就知道这么挑选，肯定是历经多次尝试。
media_cate_feature = ['app_cate_id', 'f_channel', 'app_id', 'inner_slot_id']

In [16]:
#9个与上下文内容有关的特征，一起拿出来处理。成为一个上下文特征列表。说明后面肯定会有for循环之类的迭代操作。
#注意此处忽略2个没有用到特征time，os_name 我们认为它们比较多余，将之忽略。
#亦可能不是一走上来就知道这么挑选，肯定是历经多次尝试。
content_cate_feature = ['city', 'carrier', 'province', 'nnt', 'devtype', 'osv', 'os', 'make', 'model']

In [17]:
#合并以上3个列表，成为一个总的列表，名为初始列表。说明后面肯定会有for循环之类的迭代操作。
origin_cate_list = ad_cate_feature + media_cate_feature + content_cate_feature

In [18]:
len(origin_cate_list )

26

In [19]:
#编号处理。编码，加速！！！
#这一步之前我没看懂，现在差不多明白了。就是那个初始列表毕竟只是一个只有特征名字的列表。
#那么对于列表的每个特征名字，我们要相对应地对data里对应的那个特征向量做处理，咋处理咧？
#就是建一个字典，比如'app_id'有89个可能取值，那么key 为这个可能取值的名字，比如app_id为‘298765’；value为这个id的个数，比如2900个。
#然后对每一条数据，map()该字典，所以最后总的data就变成一个只包含第多少号的int大型df.
#这个想法非常的棒，毕竟文字特别长，又难解析，还不如干脆编编号，这一删繁就简的思想学到了！
for i in origin_cate_list:
    data[i] = data[i].map(
        dict(zip(data[i].unique(), range(0, data[i].nunique()))))

In [20]:
#那data中共有39个特征向量；初始列表里只有26个特征向量；还有13条没进行这个编号处理。
data.head()

,Unnamed: 0,adid,advert_id,advert_industry_inner,advert_name,app_cate_id,app_id,app_paid,campaign_id,carrier,...,os,os_name,osv,province,time,user_tags,day,hour,label,advert_industry_inner_1
0,0,0,0,0,0,0,0,0,0,0,...,0,android,0,0,2190219034,-1,29,2,0,0
1,1,1,0,0,0,1,1,0,0,1,...,0,android,1,1,2190221070,"2100191,2100078,3001825,,3001781,3001791,30017...",29,2,0,0
2,2,2,1,1,1,2,2,0,1,1,...,0,android,2,2,2190219793,-1,29,2,0,1
3,3,3,2,2,2,2,3,0,2,2,...,0,android,3,0,2190221704,"2100098,gd_2100000,3001791,3001795,3002193,300...",29,2,0,2
4,4,4,0,0,0,3,4,0,0,0,...,1,ios,4,0,2190220024,-1,29,2,0,0


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041674 entries, 0 to 1041673
Data columns (total 39 columns):
Unnamed: 0                 1041674 non-null int64
adid                       1041674 non-null int64
advert_id                  1041674 non-null int64
advert_industry_inner      1041674 non-null int64
advert_name                1041674 non-null int64
app_cate_id                1041674 non-null int64
app_id                     1041674 non-null int64
app_paid                   1041674 non-null int64
campaign_id                1041674 non-null int64
carrier                    1041674 non-null int64
city                       1041674 non-null int64
creative_has_deeplink      1041674 non-null int64
creative_height            1041674 non-null int64
creative_id                1041674 non-null int64
creative_is_download       1041674 non-null int64
creative_is_js             1041674 non-null int64
creative_is_jump           1041674 non-null int64
creative_is_voicead        1041674 no

In [22]:
#建一个列表，用来对特征进行计数。
count_feature_list = []

In [23]:
#OK,大佬的这一段暂时没搞彻底懂，先理解如下；
def feature_count(data, features=[], is_feature=True):
    #避免相同的特征如是处理两次，这样就赘余了。
    if len(set(features)) != len(features):  #Alright,我大set()的原意就是一个无序不重复元素集.
        print('equal feature !!!!')
        return data
    #建立String类型变量new_feature，且其前缀为'count'，意为我是计数变量。
    #后面可能变为count_app_id,count_day_hour_app_id;如果里面有子字符串'add_'，则将其去除。
    new_feature = 'count'
    #建立列表nunique[]，意为我是来统计独立变量数的，比如传入的feautures列表有仨特征['day', 'hour', 'app_id'],其中day有31个取值，hour有24个取值，app_id有3888个取值。
    #那么我大nunique[]列表就长这样：[31，24，3888]
    nunique = []
    for i in features:
        nunique.append(data[i].nunique())
        new_feature += '_' + i.replace('add_', '')
    #如果传入的feautures列表有仨特征比如['day', 'hour', 'app_id']时,倘若我大nunique[]列表就长这样：[31，24，10]，说明你这特征'app_id'很奇怪，
    #就打印“不能用作交叉的特征”。
    if len(features) > 1 and len(data[features].drop_duplicates()) <= np.max(nunique):
        print(new_feature, 'is unvalid cross feature:')
        return data
    
    #建一个临时df,为data被传入的feautures列表所分组，然后聚合操作为size(),意为计数（连NaN也算进去）
    #然后重建新的index；然后把第一列的列名重命名一下，类似于count_app_id或者count_day_hour_app_id这样。
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    #与上面的临时df合并，把新的计数特征并进来，连接方式为左连接。连接键为传入的feautures列表。
    data = data.merge(temp, 'left', on=features)
  
    #如果传入的is_feature为True,意为是一个特征；那么就对def之前的那个全局变量 count_feature_list （即计数特征的列表）进行添加操作。
    if is_feature:
        count_feature_list.append(new_feature)
        
    #对于长这样（count_day_hour_app_id）的String类型变量new_feature，我们把它打印一下；并且把data.day == 3所在行的new_feature列乘以4并重新赋值；
    if 'day_' in new_feature:
        print('fix:', new_feature)
        data.loc[data.day == 3, new_feature] = data[data.day == 3][new_feature] * 4
    return data

In [24]:
#对于初始列表中的特征，若某特征有超过5个可能的取值，那么我们就对它进行feature_count处理；先是仅对该特征如此处理；
#再是对特征列表['day', 'hour',该特征]如此处理。
for i in origin_cate_list:
    n = data[i].nunique()
    if n > 5:
        data = feature_count(data, [i])
        data = feature_count(data, ['day', 'hour', i])


fix: count_day_hour_adid
fix: count_day_hour_advert_id
fix: count_day_hour_orderid
fix: count_day_hour_advert_industry_inner_1
fix: count_day_hour_advert_industry_inner
fix: count_day_hour_advert_name
fix: count_day_hour_campaign_id
fix: count_day_hour_creative_id
fix: count_day_hour_creative_tp_dnf
fix: count_day_hour_app_cate_id
fix: count_day_hour_f_channel
fix: count_day_hour_app_id
fix: count_day_hour_inner_slot_id
fix: count_day_hour_city
fix: count_day_hour_province
fix: count_day_hour_nnt
fix: count_day_hour_osv
fix: count_day_hour_make
fix: count_day_hour_model


In [25]:
data.head()

,Unnamed: 0,adid,advert_id,advert_industry_inner,advert_name,app_cate_id,app_id,app_paid,campaign_id,carrier,...,count_province,count_day_hour_province,count_nnt,count_day_hour_nnt,count_osv,count_day_hour_osv,count_make,count_day_hour_make,count_model,count_day_hour_model
0,0,0,0,0,0,0,0,0,0,0,...,68841,91,780942,1292,25473,28,151572,194,561,2
1,1,1,0,0,0,1,1,0,0,1,...,19931,23,780942,1292,72611,110,82761,131,1553,2
2,2,2,1,1,1,2,2,0,1,1,...,44633,58,780942,1292,92912,174,215457,275,1094,4
3,3,3,2,2,2,2,3,0,2,2,...,68841,91,780942,1292,149902,211,103043,314,9730,21
4,4,4,0,0,0,3,4,0,0,0,...,68841,91,60489,90,600,3,52829,198,2326,10


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041674 entries, 0 to 1041673
Data columns (total 77 columns):
Unnamed: 0                                1041674 non-null int64
adid                                      1041674 non-null int64
advert_id                                 1041674 non-null int64
advert_industry_inner                     1041674 non-null int64
advert_name                               1041674 non-null int64
app_cate_id                               1041674 non-null int64
app_id                                    1041674 non-null int64
app_paid                                  1041674 non-null int64
campaign_id                               1041674 non-null int64
carrier                                   1041674 non-null int64
city                                      1041674 non-null int64
creative_has_deeplink                     1041674 non-null int64
creative_height                           1041674 non-null int64
creative_id                               

In [27]:
#新建一个比例特征列表。
ratio_feature_list = []
#乍一看这句话还有点懵，[]中就放了一个特征名‘adid’。
#顺便，回忆一下，content_cate_feature = ['city', 'carrier', 'province', 'nnt', 'devtype', 'osv', 'os', 'make', 'model']
for i in ['adid']:
    for j in content_cate_feature:
        data = feature_count(data, [i, j])
        #这话讲得真尬，hh.当‘adid’有多于5个不同取值，且content_cate_feature中的某特征也有多于5个不同取值时，新建两个特征列；
        #一个叫i中j;一个叫j中i.这里就用到了前面得到的count_blah,blah特征向量。
        #并且在比例特征列表中添加两个string成员，一个叫i中j;一个叫j中i。
        if data[i].nunique() > 5 and data[j].nunique() > 5:
            data['ratio_' + j + '_of_' + i] = data['count_' + i + '_' + j] / data['count_' + i]
            data['ratio_' + i + '_of_' + j] = data['count_' + i + '_' + j] / data['count_' + j]
            ratio_feature_list.append('ratio_' + j + '_of_' + i)
            ratio_feature_list.append('ratio_' + i + '_of_' + j)
            print(i,'&',j)
            


adid & city
adid & province
adid & nnt
adid & osv
adid & make
adid & model


In [28]:
#同上，只不过这里i是从媒体相关特征列表media_cate_feature中取值，j是从上下文内容相关特征列表media_cate_feature以及广告相关特征ad_cate_feature中取值；
#瞬间明白上面那句并不是真的尬了，而是有意而为之。
#回忆一下，media_cate_feature = ['app_cate_id', 'f_channel', 'app_id', 'inner_slot_id']
#content_cate_feature = ['city', 'carrier', 'province', 'nnt', 'devtype', 'osv', 'os', 'make', 'model']
#ad_cate_feature = ['adid', 'advert_id', 'orderid', 'advert_industry_inner_1', 'advert_industry_inner', 'advert_name',
#                   'campaign_id', 'creative_id', 'creative_type', 'creative_tp_dnf', 'creative_has_deeplink',
#                   'creative_is_jump', 'creative_is_download']
for i in media_cate_feature:
    for j in content_cate_feature + ad_cate_feature:
        new_feature = 'inf_' + i + '_' + j
        data = feature_count(data, [i, j])
        if data[i].nunique() > 5 and data[j].nunique() > 5:
            data['ratio_' + j + '_of_' + i] = data[ 'count_' + i + '_' + j] / data['count_' + i]
            data['ratio_' + i + '_of_' + j] = data['count_' + i + '_' + j] / data['count_' + j]
            ratio_feature_list.append('ratio_' + j + '_of_' + i)
            ratio_feature_list.append('ratio_' + i + '_of_' + j)
            print(i,'&',j)

app_cate_id & city
app_cate_id & province
app_cate_id & nnt
app_cate_id & osv
app_cate_id & make
app_cate_id & model
app_cate_id & adid
app_cate_id & advert_id
app_cate_id & orderid
app_cate_id & advert_industry_inner_1
app_cate_id & advert_industry_inner
app_cate_id & advert_name
app_cate_id & campaign_id
app_cate_id & creative_id
app_cate_id & creative_tp_dnf
f_channel & city
f_channel & province
f_channel & nnt
f_channel & osv
f_channel & make
f_channel & model
f_channel & adid
f_channel & advert_id
f_channel & orderid
f_channel & advert_industry_inner_1
f_channel & advert_industry_inner
f_channel & advert_name
f_channel & campaign_id
f_channel & creative_id
f_channel & creative_tp_dnf
app_id & city
app_id & province
app_id & nnt
app_id & osv
app_id & make
app_id & model
app_id & adid
app_id & advert_id
app_id & orderid
app_id & advert_industry_inner_1
app_id & advert_industry_inner
app_id & advert_name
app_id & campaign_id
app_id & creative_id
app_id & creative_tp_dnf
inner_slot_id

In [29]:
#特征组合：将原始特征列表copy一份叫做种类特征列表cate_feature；将跟数字相关的特征、列表放到一块，起个新名字叫数字特征列表num_feature。
#再把上述特征，几乎是所涉及到了的所有特征进行大合并成新的列表，起名为feature；把它的长度及内容打印出来。

cate_feature = origin_cate_list
num_feature = ['creative_width', 'creative_height', 'hour'] + count_feature_list + ratio_feature_list
feature = cate_feature + num_feature
print(len(feature), feature)

296 ['adid', 'advert_id', 'orderid', 'advert_industry_inner_1', 'advert_industry_inner', 'advert_name', 'campaign_id', 'creative_id', 'creative_type', 'creative_tp_dnf', 'creative_has_deeplink', 'creative_is_jump', 'creative_is_download', 'app_cate_id', 'f_channel', 'app_id', 'inner_slot_id', 'city', 'carrier', 'province', 'nnt', 'devtype', 'osv', 'os', 'make', 'model', 'creative_width', 'creative_height', 'hour', 'count_adid', 'count_day_hour_adid', 'count_advert_id', 'count_day_hour_advert_id', 'count_orderid', 'count_day_hour_orderid', 'count_advert_industry_inner_1', 'count_day_hour_advert_industry_inner_1', 'count_advert_industry_inner', 'count_day_hour_advert_industry_inner', 'count_advert_name', 'count_day_hour_advert_name', 'count_campaign_id', 'count_day_hour_campaign_id', 'count_creative_id', 'count_day_hour_creative_id', 'count_creative_tp_dnf', 'count_day_hour_creative_tp_dnf', 'count_app_cate_id', 'count_day_hour_app_cate_id', 'count_f_channel', 'count_day_hour_f_channel',

In [30]:
# 低频过滤；
#对于种类特征列表cate_feature进行迭代，如果 'count_'+ feature也在data的列中，先把你这个特征打印出来再说；
#把计数值'count_' + feature为1的行数（不管你是那个特征计数值为1，只要出现了1）的这个特征赋值为-1；因为你没啥用；
#最后把这个特征值在原有基础上再加1；原---原+1；-1-----0.
for feature in cate_feature:
    if 'count_' + feature in data.keys(): #它这里不是一个DataFrame了，所以没有columns()属性了；它是一个Table,有keys()属性；
        print(feature)
        data.loc[data['count_' + feature] < 2, feature] = -1
        data[feature] = data[feature] + 1

adid
advert_id
orderid
advert_industry_inner_1
advert_industry_inner
advert_name
campaign_id
creative_id
creative_tp_dnf
app_cate_id
f_channel
app_id
inner_slot_id
city
province
nnt
osv
make
model


In [32]:
#建立一个df,名为predict，里面的每一行满足(data.label == -1) & (data.data_type == 2)；也就是说里面全是反例样本（即需要预测的样本集合）；
#那个-1就是最开始那个np.fillna('-1')d的产物。

#predict = data[(data.label == -1) & (data.data_type == 2)]
predict = data[data.label == -1]
#建立一个df,名为predict_result，它现在只有一列，就是上面predict的instance_id列。
predict_result = predict[['instance_id']]
#给predict_result添加一列，列名为'predicted_score'，并初始化为0.
predict_result['predicted_score'] = 0
##建立一个df,名为predict_x，它就是上面predict去掉'label'列之后的结果，也就是特征向量集。
predict_x = predict.drop('label', axis=1)
#建立训练集_x,而且里面全是正例样本。drop=True一定要写，意为返回一个DataFrame.
train_x = data[data.label != -1].reset_index(drop=True)
#pop()的功能就是删除并返回所删除的内容。将之传给训练集_y.
train_y = train_x.pop('label').values
#训练集_x压缩稀疏矩阵基；基差不多就就是坐标系的意思。
base_train_csr = sparse.csr_matrix((len(train_x), 0))
#预测集_x压缩稀疏矩阵基
base_predict_csr = sparse.csr_matrix((len(predict_x), 0))

In [33]:
#建一个独立编码器
enc = OneHotEncoder()
#对于种类特征列表cate_feature中的每一个特征进行独热编码并且跟之前的矩阵基进行水平方向上的合并，.values.reshape(-1, 1)这个东西我不知道是否必要。
#大神的代码就是思路清晰，并且有好多for循环。
#这个 sparse.hstack(, 'csr', 'bool')非常必要，代表粗细粒度。 
#以后再使用hstack的时候，要从粗粒度往细粒度加，如bool->int32->float32->float64,，要不然细粒度的特征就会被压缩，信息损失很多
for feature in cate_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    base_train_csr = sparse.hstack((base_train_csr, enc.transform(train_x[feature].values.reshape(-1, 1))), 'csr', 'bool')
    base_predict_csr = sparse.hstack((base_predict_csr, enc.transform(predict[feature].values.reshape(-1, 1))), 'csr', 'bool')
print('one-hot prepared !')

one-hot prepared !


In [34]:
#建一个文本特征提取器
cv = CountVectorizer(min_df=20)
#对于特征'user_tags'，仅这一条，嗯哼，你没看错；对其进行文本特征提取并且跟之前的矩阵基进行水平方向上的合并。
#.astype(str)非常有必要，因为可能它原来是obj类型，不管它以前是啥，反正这样肯定不会错。
for feature in ['user_tags']:
    data[feature] = data[feature].astype(str)
    cv.fit(data[feature])
    base_train_csr = sparse.hstack((base_train_csr, cv.transform(train_x[feature].astype(str))), 'csr', 'bool')
    base_predict_csr = sparse.hstack((base_predict_csr, cv.transform(predict_x[feature].astype(str))), 'csr', 'bool')
print('cv prepared !')

cv prepared !


In [35]:
#训练集，预测集压缩稀疏矩阵版本；
#将数字特征列表num_feature跟之前的矩阵基进行水平方向上的合并。
train_csr = sparse.hstack( (sparse.csr_matrix(train_x[num_feature]), base_train_csr), 'csr').astype('float32')
predict_csr = sparse.hstack((sparse.csr_matrix(predict_x[num_feature]), base_predict_csr), 'csr').astype('float32')

In [36]:
#建立一个lgb_model，LGBM分类器。
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=61, reg_alpha=3, reg_lambda=1,
    max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.8, colsample_bytree=0.8, subsample_freq=1,
    learning_rate=0.035, random_state=2018, n_jobs=10
)
#建立一个分层K折采样器。为5折。
skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)

In [37]:
best_score = []
for index, (train_index, test_index) in enumerate(skf.split(train_csr, train_y)):
    lgb_model.fit(train_csr[train_index], train_y[train_index],
                  eval_set=[(train_csr[train_index], train_y[train_index]),
                            (train_csr[test_index], train_y[test_index])], early_stopping_rounds=200, verbose=10)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    test_pred = lgb_model.predict_proba(predict_csr, num_iteration=lgb_model.best_iteration_)[:, 1]
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
predict_result['predicted_score'] = predict_result['predicted_score'] / 5
mean = predict_result['predicted_score'].mean()
print('mean:', mean)

Training until validation scores don't improve for 200 rounds.
[10]	valid_0's binary_logloss: 0.464251	valid_1's binary_logloss: 0.464136
[20]	valid_0's binary_logloss: 0.445588	valid_1's binary_logloss: 0.44551
[30]	valid_0's binary_logloss: 0.43457	valid_1's binary_logloss: 0.434587
[40]	valid_0's binary_logloss: 0.427808	valid_1's binary_logloss: 0.427971
[50]	valid_0's binary_logloss: 0.423535	valid_1's binary_logloss: 0.423857
[60]	valid_0's binary_logloss: 0.420688	valid_1's binary_logloss: 0.421164
[70]	valid_0's binary_logloss: 0.41879	valid_1's binary_logloss: 0.419494
[80]	valid_0's binary_logloss: 0.417454	valid_1's binary_logloss: 0.418334
[90]	valid_0's binary_logloss: 0.416481	valid_1's binary_logloss: 0.417578
[100]	valid_0's binary_logloss: 0.415738	valid_1's binary_logloss: 0.417046
[110]	valid_0's binary_logloss: 0.41513	valid_1's binary_logloss: 0.416663
[120]	valid_0's binary_logloss: 0.414597	valid_1's binary_logloss: 0.416337
[130]	valid_0's binary_logloss: 0.4141

[140]	valid_0's binary_logloss: 0.413711	valid_1's binary_logloss: 0.416354
[150]	valid_0's binary_logloss: 0.413318	valid_1's binary_logloss: 0.416203
[160]	valid_0's binary_logloss: 0.412956	valid_1's binary_logloss: 0.416081
[170]	valid_0's binary_logloss: 0.412607	valid_1's binary_logloss: 0.415957
[180]	valid_0's binary_logloss: 0.41228	valid_1's binary_logloss: 0.415867
[190]	valid_0's binary_logloss: 0.411964	valid_1's binary_logloss: 0.415766
[200]	valid_0's binary_logloss: 0.411663	valid_1's binary_logloss: 0.415709
[210]	valid_0's binary_logloss: 0.411361	valid_1's binary_logloss: 0.415652
[220]	valid_0's binary_logloss: 0.411066	valid_1's binary_logloss: 0.415591
[230]	valid_0's binary_logloss: 0.410785	valid_1's binary_logloss: 0.415535
[240]	valid_0's binary_logloss: 0.410505	valid_1's binary_logloss: 0.415487
[250]	valid_0's binary_logloss: 0.410232	valid_1's binary_logloss: 0.415446
[260]	valid_0's binary_logloss: 0.409963	valid_1's binary_logloss: 0.415408
[270]	valid_0

[60]	valid_0's binary_logloss: 0.420763	valid_1's binary_logloss: 0.421014
[70]	valid_0's binary_logloss: 0.418865	valid_1's binary_logloss: 0.419243
[80]	valid_0's binary_logloss: 0.417563	valid_1's binary_logloss: 0.418093
[90]	valid_0's binary_logloss: 0.416598	valid_1's binary_logloss: 0.417285
[100]	valid_0's binary_logloss: 0.415849	valid_1's binary_logloss: 0.416692
[110]	valid_0's binary_logloss: 0.415231	valid_1's binary_logloss: 0.416263
[120]	valid_0's binary_logloss: 0.414712	valid_1's binary_logloss: 0.415962
[130]	valid_0's binary_logloss: 0.414256	valid_1's binary_logloss: 0.415724
[140]	valid_0's binary_logloss: 0.413843	valid_1's binary_logloss: 0.415531
[150]	valid_0's binary_logloss: 0.413453	valid_1's binary_logloss: 0.415376
[160]	valid_0's binary_logloss: 0.413098	valid_1's binary_logloss: 0.41526
[170]	valid_0's binary_logloss: 0.412741	valid_1's binary_logloss: 0.415153
[180]	valid_0's binary_logloss: 0.41241	valid_1's binary_logloss: 0.415044
[190]	valid_0's bi

[170]	valid_0's binary_logloss: 0.412699	valid_1's binary_logloss: 0.415531
[180]	valid_0's binary_logloss: 0.412368	valid_1's binary_logloss: 0.415425
[190]	valid_0's binary_logloss: 0.412056	valid_1's binary_logloss: 0.415346
[200]	valid_0's binary_logloss: 0.411742	valid_1's binary_logloss: 0.415257
[210]	valid_0's binary_logloss: 0.411442	valid_1's binary_logloss: 0.41518
[220]	valid_0's binary_logloss: 0.411153	valid_1's binary_logloss: 0.415118
[230]	valid_0's binary_logloss: 0.410863	valid_1's binary_logloss: 0.415054
[240]	valid_0's binary_logloss: 0.410575	valid_1's binary_logloss: 0.414995
[250]	valid_0's binary_logloss: 0.410292	valid_1's binary_logloss: 0.414939
[260]	valid_0's binary_logloss: 0.410028	valid_1's binary_logloss: 0.414904
[270]	valid_0's binary_logloss: 0.409769	valid_1's binary_logloss: 0.414861
[280]	valid_0's binary_logloss: 0.409502	valid_1's binary_logloss: 0.414828
[290]	valid_0's binary_logloss: 0.409251	valid_1's binary_logloss: 0.4148
[300]	valid_0's

[320]	valid_0's binary_logloss: 0.408268	valid_1's binary_logloss: 0.415849
[330]	valid_0's binary_logloss: 0.408028	valid_1's binary_logloss: 0.415827
[340]	valid_0's binary_logloss: 0.407788	valid_1's binary_logloss: 0.415811
[350]	valid_0's binary_logloss: 0.407545	valid_1's binary_logloss: 0.415783
[360]	valid_0's binary_logloss: 0.407304	valid_1's binary_logloss: 0.415767
[370]	valid_0's binary_logloss: 0.407083	valid_1's binary_logloss: 0.415754
[380]	valid_0's binary_logloss: 0.406841	valid_1's binary_logloss: 0.415731
[390]	valid_0's binary_logloss: 0.406612	valid_1's binary_logloss: 0.415712
[400]	valid_0's binary_logloss: 0.406385	valid_1's binary_logloss: 0.415684
[410]	valid_0's binary_logloss: 0.406159	valid_1's binary_logloss: 0.415681
[420]	valid_0's binary_logloss: 0.405937	valid_1's binary_logloss: 0.415663
[430]	valid_0's binary_logloss: 0.405722	valid_1's binary_logloss: 0.41566
[440]	valid_0's binary_logloss: 0.405503	valid_1's binary_logloss: 0.415656
[450]	valid_0

In [39]:
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')
predict_result[['instance_id', 'predicted_score']].to_csv(path + "/submission_lgb_baseline_%s.csv" % now, index=False)